# 📚 Kapitola 20: Projekt 2 - REST API pro blog

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 2 | INTERMEDIATE</h2>
    <p style="color: white; margin: 10px 0;">🎯 PROJEKT</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na následující témata:

- **FastAPI application structure**
- **User registration/authentication**
- **Blog posts CRUD**
- **Comments system**
- **Pagination a filtering**
- **File uploads**
- **API testing with pytest**

## ⚠️ Předpoklady
Tato kapitola navazuje na kapitoly: 11, 12, 13, 15, 16, 17, 18, 19

---


In [ ]:
# ⚙️ Inicializace prostředí
# Tento kód nastavuje prostředí pro kapitolu 20

import sys
import os
from datetime import datetime

# Informace o prostředí
print("=" * 60)
print(f"📚 KAPITOLA 20: Projekt 2 - REST API pro blog")
print("=" * 60)
print(f"🐍 Python verze: {sys.version}")
print(f"📅 Datum spuštění: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 OS: {os.name}")
print(f"📁 Pracovní adresář: {os.getcwd()}")
print("=" * 60)

# Instalace potřebných knihoven (odkomentujte podle potřeby)
# !pip install requests pandas numpy matplotlib
# !pip install beautifulsoup4 sqlalchemy fastapi

# Import základních knihoven
import json
import csv
import time
import random
from pathlib import Path
from typing import List, Dict, Optional, Any

print("✅ Prostředí připraveno!")

## 📖 Teoretická část

<div style="background: #f0f4ff; padding: 20px; border-left: 5px solid #4a69bd; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🎓 Základní teorie a koncepty</h3>
</div>

# Projekt 2 - REST API pro blog

## 1. Úvod a motivace

REST API je základní stavební kámen moderních webových aplikací. V tomto projektu se naučíš vytvářet plnohodnotnou REST API aplikaci pro blog pomocí FastAPI – moderního a rychlého Python frameworku. Tento projekt je důležitý, protože ukazuje, jak propojit backend s frontendem, zajišťovat autentizaci a spravovat komplexní datové struktury.

V praxi se REST API používá všude – od webových aplikací přes mobilní aplikace až po integrace mezi různými systémy. Například sociální sítě, e-commerce platformy, portály a školní systémy používají REST API pro komunikaci mezi klientem a serverem.

Při práci na tomto projektu se naučíš:
- Vytvářet strukturované FastAPI aplikace
- Pracovat s autentizací uživatelů pomocí JWT tokenů
- Implementovat systém pro správu článků a komentářů
- Nastavovat filtraci, stránkování a nahrávání souborů
- Testovat API pomocí pytest

## 2. Hlavní koncepty

### FastAPI application structure

FastAPI je moderní, rychlý a snadno testovatelný framework pro tvorbu REST API v Pythonu. Struktura aplikace se skládá z několika klíčových součástí:

- **Routes** – definují endpointy API (např. `/users`, `/posts`)
- **Models** – představují datové struktury (pydantic models)
- **Database connections** – připojení k databázi
- **Dependency injection** – správa závislostí mezi komponentami
- **Security** – autentizace a autorizace

Příklad základní struktury:
```python
from fastapi import FastAPI, Depends
from pydantic import BaseModel

app = FastAPI()

class User(BaseModel):
    id: int
    name: str
    email: str

@app.get("/users/{user_id}")
async def get_user(user_id: int):
    return {"id": user_id, "name": "Jan Novák"}
```

### User registration/authentication

Autentizace uživatelů je základní součástí každé moderní aplikace. V tomto projektu použijeme JWT tokeny pro bezpečné přihlašování.

Klíčové komponenty:
- **Hashování hesel** pomocí `bcrypt`
- **JWT tokeny** pomocí `python-jose`
- **Dependency injection** pro kontrolu přístupu

```python
from passlib.context import CryptContext
from jose import JWTError, jwt
from datetime import datetime, timedelta

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

def verify_password(plain_password, hashed_password):
    return pwd_context.verify(plain_password, hashed_password)

def get_password_hash(password):
    return pwd_context.hash(password)
```

### Blog posts CRUD

CRUD (Create, Read, Update, Delete) operace jsou základní pro správu obsahu. Pro blogy to znamená:
- Vytváření nových článků
- Čtení článků (včetně detailu)
- Aktualizace článků
- Odstraňování článků

```python
@app.post("/posts/")
async def create_post(post: PostCreate, db: Session = Depends(get_db)):
    db_post = models.Post(**post.dict())
    db.add(db_post)
    db.commit()
    db.refresh(db_post)
    return db_post
```

### Comments system

Systém komentářů umožňuje interakci mezi uživateli a obsahem. Každý komentář má:
- Odkaz na článek (`post_id`)
- Referenci na autora (`user_id`)
- Datum vytvoření
- Text komentáře

```python
class Comment(BaseModel):
    id: int
    post_id: int
    user_id: int
    content: str
    created_at: datetime
```

### Pagination a filtering

Pro velké množství dat je důležité umožnit stránkování a filtrování:

```python
@app.get("/posts/")
async def get_posts(skip: int = 0, limit: int = 10, category: str = None):
    query = db.query(models.Post)
    if category:
        query = query.filter(models.Post.category == category)
    posts = query.offset(skip).limit(limit).all()
    return posts
```

### File uploads

Nahrávání souborů umožňuje uživatelům přidávat obrázky k článkům:

```python
from fastapi import UploadFile, File

@app.post("/posts/{post_id}/upload-image")
async def upload_image(post_id: int, file: UploadFile = File(...)):
    # Uložení souboru
    contents = await file.read()
    # Zpracování obrázku a uložení cesty do databáze
    return {"filename": file.filename}
```

### API testing with pytest

Testování je klíčové pro zajištění kvality API. Použijeme `pytest` spolu s `httpx`:

```python
import pytest
from httpx import AsyncClient
from main import app

@pytest.mark.asyncio
async def test_create_post():
    async with AsyncClient(app=app, base_url="http://test") as ac:
        response = await ac.post("/posts/", json={"title": "Test", "content": "Content"})
        assert response.status_code == 200
```

## 3. Důležité detaily

### Časté chyby a jak se jim vyhnout

- **Nesprávná práce s databází** – vždy používej `Session` jako závislost
- **Chybné validace dat** – využívej Pydantic modely pro automatickou validaci
- **Neefektivní dotazy** – používej `select()` místo `query()` k optimalizaci

### Best practices

- Vždy používej typové anotace (type hints)
- Implementuj logging a error handling
- Používej middleware pro zpracování požadavků (např. CORS)
- Dělaj jednu věc na jednom endpointu (single responsibility principle)

### Performance tipy

- Využívej `async`/`await` pro neblokující operace
- Implementuj cacheování pro často požadovaná data
- Používej indexy v databázi
- Optimalizuj SQL dotazy (avoid N+1 problem)

## 4. Propojení s předchozími kapitolami

Tento projekt využívá znalosti z předchozích kapitol jako jsou:
- Základy Pythonu a REST API
- Práce s databází (SQLAlchemy)
- Autentizace a bezpečnost

Rozšiřuje již naučené dovednosti o pokročilé funkce jako je testování, stránkování a práce s komplexními datovými strukturami. Vytvoříš kompletní systém, který můžeš použít jako základ pro reálné projekty.


## 💻 Praktické příklady

<div style="background: #e8f5e9; padding: 20px; border-left: 5px solid #4caf50; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">👨‍💻 Hands-on příklady ke spuštění</h3>
    <p>Následující příklady si můžete hned vyzkoušet. Každý příklad je samostatně spustitelný.</p>
</div>


### Příklad 1

Příklad 1: Základní FastAPI aplikace

### Příklad 2

### Co příklad demonstruje:
Založení základní FastAPI aplikace s jednoduchým endpointem pro uvítání uživatele.

In [ ]:
from fastapi import FastAPI

# Vytvoření instance FastAPI aplikace
app = FastAPI(title="Blog API", version="1.0.0")

# Definice jednoduchého endpointu
@app.get("/")
async def root():
    """
    Hlavní endpoint aplikace, který vrací uvítací zprávu
    """
    return {"message": "Vítejte v blogové API!"}

# Endpoint pro získání informací o aplikaci
@app.get("/info")
async def get_info():
    """
    Vrací informace o aplikaci
    """
    return {
        "name": "Blog API",
        "version": "1.0.0",
        "description": "REST API pro správu blogových příspěvků"
    }

# Spuštění aplikace: uvědomte si, že pro spuštění použijte příkaz:
# uvicorn main:app --reload

### Příklad 3

### Očekávaný výstup:
Při spuštění aplikace a přístupu na `http://localhost:8000/`:
```json
{
  "message": "Vítejte v blogové API!"
}
```

Při přístupu na `http://localhost:8000/info`:
```json
{
  "name": "Blog API",
  "version": "1.0.0",
  "description": "REST API pro správu blogových příspěvků"
}
```

### Příklad 4

### Vysvětlení fungování:
Tento kód vytváří základní FastAPI aplikaci s dvěma endpointy. První (`/`) vrací uvítací zprávu, druhý (`/info`) poskytuje informace o aplikaci. FastAPI automaticky generuje interaktivní dokumentaci na adrese `http://localhost:8000/docs` při spuštění s přepínačem `--reload`.

---

##

### Příklad 5

Příklad 2: Registrace a autentifikace uživatele

### Příklad 6

### Co příklad demonstruje:
Základní registraci nového uživatele a jednoduchou autentifikaci pomocí tokenů.

In [ ]:
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel
from passlib.context import CryptContext
import jwt
from datetime import datetime, timedelta

# Inicializace aplikace
app = FastAPI()

# Konfigurace šifrování hesel
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# Model pro registraci uživatele
class UserRegister(BaseModel):
    username: str
    email: str
    password: str

# Model pro přihlášení uživatele
class UserLogin(BaseModel):
    username: str
    password: str

# Simulovaná databáze uživatelů (v reálné aplikaci by to byla skutečná databáze)
fake_users_db = {}

# Generování JWT tokenu
SECRET_KEY = "my_secret_key"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

def create_access_token(data: dict):
    """Vytvoření přístupového tokenu pro uživatele"""
    to_encode = data.copy()
    expire = datetime.utcnow() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt

@app.post("/register")
async def register_user(user: UserRegister):
    """
    Registrace nového uživatele
    """
    # Kontrola, zda uživatel již existuje
    if user.username in fake_users_db:
        raise HTTPException(status_code=400, detail="Uživatel s tímto jménem již existuje")
    
    # Šifrování hesla a uložení uživatele
    hashed_password = pwd_context.hash(user.password)
    fake_users_db[user.username] = {
        "username": user.username,
        "email": user.email,
        "hashed_password": hashed_password
    }
    
    return {"message": "Uživatel úspěšně zaregistrován"}

@app.post("/login")
async def login_user(user: UserLogin):
    """
    Přihlášení uživatele a vytvoření přístupového tokenu
    """
    # Hledání uživatele v databázi
    db_user = fake_users_db.get(user.username)
    if not db_user:
        raise HTTPException(status_code=401, detail="Špatné uživatelské jméno nebo heslo")
    
    # Kontrola správnosti hesla
    if not pwd_context.verify(user.password, db_user["hashed_password"]):
        raise HTTPException(status_code=401, detail="Špatné uživatelské jméno nebo heslo")
    
    # Vytvoření přístupového tokenu
    access_token = create_access_token(data={"sub": user.username})
    
    return {
        "access_token": access_token,
        "token_type": "bearer"
    }

# Spuštění aplikace: uvicorn main:app --reload

### Příklad 7

### Očekávaný výstup:
Při registraci uživatele:
```json
{
  "message": "Uživatel úspěšně zaregistrován"
}
```

Při přihlášení:
```json
{
  "access_token": "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9...",
  "token_type": "bearer"
}
```

### Příklad 8

### Vysvětlení fungování:
Tento kód implementuje základní registraci a přihlášení uživatele. Používá knihovnu `passlib` pro šifrování hesel pomocí bcrypt, a `PyJWT` pro generování JWT tokenů. Při registraci se heslo šifruje a uživatel je uložen do simulované databáze. Při přihlášení se kontroluje správnost hesla a vytvoří se přístupový token, který může být použit pro autentizaci v dalších požadavcích.

---

##

### Příklad 9

Příklad 3: CRUD operace pro blogové příspěvky

### Příklad 10

### Co příklad demonstruje:
Základní CRUD operace (vytvoření, čtení, aktualizace, mazání) pro blogové příspěvky s autentizací.

In [ ]:
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel
from typing import List, Optional
from datetime import datetime
import uuid

# Inicializace aplikace
app = FastAPI()

# Model pro příspěvek
class BlogPost(BaseModel):
    id: str
    title: str
    content: str
    author: str
    created_at: datetime
    updated_at: datetime

# Model pro vytvoření nového příspěvku
class BlogPostCreate(BaseModel):
    title: str
    content: str

# Simulovaná databáze příspěvků
fake_posts_db = {}

# Přihlášený uživatel (simulace autentizace)
current_user = "testuser"

@app.post("/posts", response_model=BlogPost)
async def create_post(post: BlogPostCreate):
    """
    Vytvoření nového blogového příspěvku
    """
    # Generování unikátního ID pro příspěvek
    post_id = str(uuid.uuid4())
    
    # Vytvoření nového příspěvku
    new_post = BlogPost(
        id=post_id,
        title=post.title,
        content=post.content,
        author=current_user,
        created_at=datetime.now(),
        updated_at=datetime.now()
    )
    
    # Uložení do databáze
    fake_posts_db[post_id] = new_post
    
    return new_post

@app.get("/posts", response_model=List[BlogPost])
async def read_posts():
    """
    Získání seznamu všech příspěvků
    """
    return list(fake_posts_db.values())

@app.get("/posts/{post_id}", response_model=BlogPost)
async def read_post(post_id: str):
    """
    Získání konkrétního příspěvku podle ID
    """
    if post_id not in fake_posts_db:
        raise HTTPException(status_code=404, detail="Příspěvek nenalezen")
    
    return fake_posts_db[post_id]

@app.put("/posts/{post_id}", response_model=BlogPost)
async def update_post(post_id: str, post_update: BlogPostCreate):
    """
    Aktualizace existujícího příspěvku
    """
    if post_id not in fake_posts_db:
        raise HTTPException(status_code=404, detail="Příspěvek nenalezen")
    
    # Aktualizace příspěvku
    updated_post = fake_posts_db[post_id]
    updated_post.title = post_update.title
    updated_post.content = post_update.content
    updated_post.updated_at = datetime.now()
    
    return updated_post

@app.delete("/posts/{post_id}")
async def delete_post(post_id: str):
    """
    Smazání příspěvku podle ID
    """
    if post_id not in fake_posts_db:
        raise HTTPException(status_code=404, detail="Příspěvek nenalezen")
    
    del fake_posts_db[post_id]
    return {"message": "Příspěvek byl smazán"}

# Spuštění aplikace: uvicorn main:app --reload

### Příklad 11

### Očekávaný výstup:
Při vytvoření příspěvku:
```json
{
  "id": "123e4567-e89b-12d3-a456-426614174000",
  "title": "Můj první příspěvek",
  "content": "Toto je obsah mého příspěvku.",

## 🎯 Cvičení a úkoly

<div style="background: #fff3e0; padding: 20px; border-left: 5px solid #ff9800; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">✍️ Praktická cvičení k procvičení</h3>
    <p>Vyřešte následující úkoly. Začněte od jednoduššších a postupujte k složitějším.</p>
</div>

# Cvičení pro Projekt 2 - REST API pro blog

---

## **Cvičení 1: Vytvoření základního REST endpointu pro přidání nového článku**

### **1. Název úkolu**
Založení REST endpointu pro přidání nového článku do blogu

### **2. Detailní zadání**
Vytvoř REST API endpoint, který umožní přidat nový článek do blogu. Endpoint musí podporovat HTTP metodu POST a očekávat JSON tělo s názvem, obsahem a autorem článku. Přidej validaci vstupních dat.

### **3. Vstupní data/požadavky**
- HTTP metoda: `POST`
- Endpoint: `/api/articles`
- JSON tělo:
  ```json
  {
    "title": "Jak začít s REST API",
    "content": "REST API je moderní způsob, jak komunikovat mezi aplikacemi...",
    "author": "Jan Novák"
  }
  ```

### **4. Očekávaný výstup**
- Vytvořený článek s automaticky generovaným ID a datem vytvoření
- Vrácený HTTP kód 201 (Created)
- JSON odpověď s vytvořeným článkem

### **5. Hints/Nápověda**
1. Použij `express` a `body-parser` pro zpracování HTTP požadavků
2. Vytvoř jednoduchou datovou strukturu pro článek (např. objekt s title, content, author, id, createdAt)
3. Implementuj validaci: všechna pole jsou povinná a title má minimální délku 5 znaků
4. Přidej jednoduché generování ID (např. `Date.now()` nebo UUID)

### **6. Kostra řešení**
```javascript
const express = require('express');
const app = express();
app.use(express.json());

let articles = [];
const generateId = () => Date.now().toString();

app.post('/api/articles', (req, res) => {
  // TODO: Validace a přidání článku
});

app.listen(3000);
```

### **7. Bonusové rozšíření**
- Přidej možnost nastavit datum publikování vlastním způsobem (např. timestamp)
- Ulož články do souboru nebo do databáze (např. SQLite)

---

## **Cvičení 2: Implementace GET endpointu pro získání článku podle ID**

### **1. Název úkolu**
Získání jednoho článku podle ID

### **2. Detailní zadání**
Vytvoř endpoint, který umožní získat konkrétní článek podle jeho ID. Pokud článek neexistuje, vrať HTTP kód 404.

### **3. Vstupní data/požadavky**
- HTTP metoda: `GET`
- Endpoint: `/api/articles/:id`

### **4. Očekávaný výstup**
- Pokud článek existuje, vratí se HTTP kód 200 a JSON s článkem
- Pokud neexistuje, vrať HTTP kód 404

### **5. Hints/Nápověda**
1. Použij `req.params.id` pro získání ID z URL
2. Vytvoř funkci pro vyhledávání článku podle ID v poli
3. Zkontroluj, jestli článek existuje před jeho vrácením
4. Přidej ošetření chyby pro nevalidní ID

### **6. Kostra řešení**
```javascript
app.get('/api/articles/:id', (req, res) => {
  // TODO: Vyhledání a vrácení článku podle ID
});

const findArticleById = (id) => {
  // TODO: Implementace vyhledávání
};
```

### **7. Bonusové rozšíření**
- Přidej možnost filtrovat články podle autora nebo data vytvoření
- Přidej stránkování pro získání více článků najednou

---

## **Cvičení 3: Aktualizace a mazání článku**

### **1. Název úkolu**
Implementace PUT a DELETE endpointů pro články

### **2. Detailní zadání**
Vytvoř endpointy pro aktualizaci (`PUT`) a mazání (`DELETE`) článku podle ID. Zajištění správného HTTP kódu odpovědi.

### **3. Vstupní data/požadavky**
- PUT `/api/articles/:id` - aktualizace celého článku
- DELETE `/api/articles/:id` - smazání článku

### **4. Očekávaný výstup**
- PUT: Vrátí HTTP kód 200 a aktualizovaný článek
- DELETE: Vrátí HTTP kód 204 (No Content) nebo 200 s potvrzením

### **5. Hints/Nápověda**
1. Použij `req.params.id` pro identifikaci článku
2. Implementuj aktualizaci celého článku nebo jen části (použij Partial<T>)
3. Při mazání ověř, že článek existuje a pak ho odstraň z pole
4. Vrať odpovídající HTTP kód v každém případě

### **6. Kostra řešení**
```javascript
app.put('/api/articles/:id', (req, res) => {
  // TODO: Aktualizace článku
});

app.delete('/api/articles/:id', (req, res) => {
  // TODO: Smazání článku
});
```

### **7. Bonusové rozšíření**
- Přidej možnost částečné aktualizace pomocí `PATCH`
- Implementuj trvalé odstranění z databáze místo jen v paměti

---

## **Cvičení 4: Vytvoření API pro získání seznamu článků s možností filtrování**

### **1. Název úkolu**
Výpis článků s filtry a stránkováním

### **2. Detailní zadání**
Implementuj endpoint, který vrátí seznam článků s podporou možností filtrování podle autora a stránkování.

### **3. Vstupní data/požadavky**
- HTTP metoda: `GET`
- Endpoint: `/api/articles?author=Jan&limit=5&page=1`

### **4. Očekávaný výstup**
- Vrátí seznam článků, které odpovídají filtru
- Vrátí počet celkových článků a informace o stránkování

### **5. Hints/Nápověda**
1. Použij `req.query` pro zpracování parametrů z URL
2. Implementuj filtrování podle autora (případně jiných položek)
3. Přidej stránkování pomocí limit a offset
4. Vrať metadata o stránkování v odpovědi

### **6. Kostra řešení**
```javascript
app.get('/api/articles', (req, res) => {
  // TODO: Implementace filtru a stránkování
});
```

### **7. Bonusové rozšíření**
- Přidej podporu pro řazení (`sort=created_at` nebo `sort=title`)
- Implementuj možnost vyhledávání podle obsahu článku

---

## **Cvičení 5: Ošetření chyb a validace dat**

### **1. Název úkolu**
Implementace globálního error handler


## 🚀 Kompletní řešení projektu

<div style="background: #f3e5f5; padding: 20px; border-left: 5px solid #9c27b0; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">💡 Referenční implementace</h3>
    <p>Zde je kompletní řešení projektu. Použijte ho jako referenci nebo inspiraci.</p>
</div>

# Projekt: REST API pro blog

Tento projekt představuje komplexní řešení REST API pro blog, které umožňuje správu článků a uživatelů. Implementace je založena na Pythonu s využitím FastAPI a SQLAlchemy.

## 1. Kompletní struktura projektu

```
blog_api/
├── app/
│   ├── __init__.py
│   ├── main.py
│   ├── api/
│   │   ├── __init__.py
│   │   ├── v1/
│   │   │   ├── __init__.py
│   │   │   ├── routes/
│   │   │   │   ├── __init__.py
│   │   │   │   ├── posts.py
│   │   │   │   └── users.py
│   │   │   └── schemas/
│   │   │       ├── __init__.py
│   │   │       ├── post.py
│   │   │       ├── user.py
│   │   │       └── token.py
│   │   └── dependencies.py
│   ├── core/
│   │   ├── __init__.py
│   │   ├── config.py
│   │   ├── database.py
│   │   └── security.py
│   ├── models/
│   │   ├── __init__.py
│   │   ├── base.py
│   │   ├── post.py
│   │   └── user.py
│   └── utils/
│       ├── __init__.py
│       └── helpers.py
├── tests/
│   ├── __init__.py
│   ├── test_posts.py
│   ├── test_users.py
│   └── conftest.py
├── alembic/
│   ├── versions/
│   │   └── 001_initial_migration.py
│   ├── env.py
│   ├── script.py.mako
│   └── README
├── requirements.txt
├── .env
├── README.md
└── pyproject.toml
```

## 2. Hlavní soubor s kódem (main.py)

```python
"""
Main application file for the blog API
"""

from fastapi import FastAPI, HTTPException, status, Depends
from fastapi.middleware.cors import CORSMiddleware
from sqlalchemy.orm import Session

from app.core.config import settings
from app.core.database import engine, get_db
from app.models.base import Base
from app.api.v1.routes import posts, users
from app.core.security import get_current_user
from app.models.user import User

# Vytvoření databázové struktury
Base.metadata.create_all(bind=engine)

# Inicializace FastAPI aplikace
app = FastAPI(
    title=settings.PROJECT_NAME,
    openapi_url=f"{settings.API_V1_STR}/openapi.json"
)

# Nastavení CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=settings.ALLOWED_ORIGINS,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Přidání routů
app.include_router(
    posts.router,
    prefix=f"{settings.API_V1_STR}/posts",
    tags=["posts"]
)
app.include_router(
    users.router,
    prefix=f"{settings.API_V1_STR}/users",
    tags=["users"]
)

@app.get("/")
async def root():
    """Hlavní endpoint aplikace"""
    return {"message": "Welcome to Blog API"}

@app.get("/health")
async def health_check():
    """Kontrola stavu aplikace"""
    return {"status": "healthy"}

@app.get("/docs")
async def documentation():
    """Zobrazí dokumentaci API"""
    return {"message": "API Documentation available at /docs"}

# Přidání základního uživatele pro testování
@app.on_event("startup")
async def create_initial_user():
    """Vytvoření počátečního uživatele při spuštění aplikace"""
    db = next(get_db())
    try:
        # Zkontrolujeme, zda uživatel již existuje
        user = db.query(User).filter(User.email == "admin@example.com").first()
        if not user:
            # Vytvoříme admina s heslem "password"
            from app.core.security import get_password_hash
            new_user = User(
                email="admin@example.com",
                username="admin",
                hashed_password=get_password_hash("password"),
                is_active=True,
                is_superuser=True
            )
            db.add(new_user)
            db.commit()
            db.refresh(new_user)
    except Exception as e:
        print(f"Chyba při vytváření počátečního uživatele: {e}")
    finally:
        db.close()

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
```

## 3. Pomocné moduly

### app/core/config.py
```python
"""
Konfigurační soubor aplikace
"""

from pydantic_settings import BaseSettings
from typing import List

class Settings(BaseSettings):
    PROJECT_NAME: str = "Blog API"
    API_V1_STR: str = "/api/v1"
    SECRET_KEY: str = "your-secret-key-here"
    ALGORITHM: str = "HS256"
    ACCESS_TOKEN_EXPIRE_MINUTES: int = 30
    DATABASE_URL: str = "sqlite:///./blog.db"
    ALLOWED_ORIGINS: List[str] = ["*"]
    
    class Config:
        env_file = ".env"

settings = Settings()
```

### app/core/database.py
```python
"""
Databázové připojení a session management
"""

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

from app.core.config import settings

# Vytvoření databázového engine
engine = create_engine(
    settings.DATABASE_URL, 
    connect_args={"check_same_thread": False} if "sqlite" in settings.DATABASE_URL else {}
)

# Vytvoření session factory
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# BaseModel pro ORM modely
Base = declarative_base()

def get_db():
    """
    Dependency pro získání databázové session
    """
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
```

### app/core/security.py
```python
"""
Bezpečnostní modul pro autentizaci a hashování hesel
"""

from datetime import datetime, timedelta
from typing import Optional
from jose import JWTError, jwt
from passlib.context import CryptContext
from fastapi import HTTPException, status, Depends
from fastapi.security import OAuth2PasswordBearer

from app.core.config import settings
from app.models.user import User

# Inicializace password context pro hashování
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# OAuth2 schema pro tokeny
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="api/v1/users/login")

def verify_password(plain_password: str, hashed_password: str) -> bool:
    """
    Ověří zadané heslo s hashem
    """
    return pwd_context.verify(plain_password, hashed_password)

def get_password_hash(password: str) -> str:
    """
    Hashuje heslo pomocí bcrypt
    """
    return pwd_context.hash(password)

def create_access_token(data: dict, expires_delta: Optional[timedelta] = None):
    """
    Vytvoří JWT token pro autentizaci
    """
    to_encode = data.copy()
    
    if expires_delta:
        expire = datetime.utcnow() + expires_delta
    else:
        expire = datetime.utcnow() + timedelta(minutes=15)
        
    to_encode.update({"exp": expire})
    
    encoded_jwt = jwt.encode(to_encode, settings.SECRET_KEY, algorithm=settings.ALGORITHM)
    return encoded_jwt

def get_current_user(token: str = Depends(oauth2_scheme), db = Depends(get_db)):
    """
    Získá aktuálního uživatele z JWT tokenu
    """
    credentials_exception = HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Could not validate credentials",
        headers={"WWW-Authenticate": "Bearer"},
    )
    
    try:
        payload = jwt.decode(token, settings.SECRET_KEY, algorithms=[settings.ALGORITHM])
        email: str = payload.get("sub")
        
        if email is None:
            raise credentials_exception
            
    except JWTError:
        raise credentials_exception
    
    user = db.query(User).filter(User.email == email).first()
    
    if user is None:
        raise credentials_exception
        
    return user
```

### app/models/base.py
```python
"""
Základní model pro všechny ORM entity
"""

from sqlalchemy import Column, DateTime
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime

Base = declarative_base()

class BaseModel(Base):
    """
    Základní model s běžnými sloupci
    """
    __abstract__ = True
    
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
```

### app/models/post.py
```python
"""
Model pro články blogu
"""

from sqlalchemy import Column, Integer, String, Text, ForeignKey, Boolean
from sqlalchemy.orm import relationship

from app.models.base import BaseModel
from app.models.user import User

class Post(BaseModel):
    """
    Model článku
    """
    __tablename__ = "posts"
    
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String(255), nullable=False)
    content = Column(Text, nullable=False)
    is_published = Column(Boolean, default=False)
    author_id = Column(Integer, ForeignKey("users.id"), nullable=False)
    
    # Vztah k uživateli
    author = relationship("User", back_populates="posts")
    
    def __repr__(self):
        return f"<Post(id={self.id}, title='{self.title}')>"
```

### app/models/user.py
```python
"""
Model pro uživatele blogu
"""

from sqlalchemy import Column, Integer, String, Boolean
from sqlalchemy.orm import relationship

from app.models.base import BaseModel

class User(BaseModel):
    """
    Model uživatele
    """
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True, index=True)
    email = Column(String(255), unique=True, index=True, nullable=False)
    username = Column(String(100), unique=True, index=True, nullable=False)
    hashed_password = Column(String(255), nullable=False)
    is_active = Column(Boolean, default=True)
    is_superuser = Column(Boolean, default=False)
    
    # Vztah k článkům
    posts = relationship("Post", back_populates="author")
    
    def __repr__(self):
        return f"<User(id={self.id}, email='{self.email}')>"
```

### app/api/v1/routes/posts.py
```python
"""
Routy pro správu článků
"""

from fastapi import APIRouter, HTTPException, status, Depends
from sqlalchemy.orm import Session

from app.core.database import get_db
from app.models.post import Post
from app.models.user import User
from app.api.v1.schemas.post import PostCreate, PostUpdate, PostResponse
from app.core.security import get_current_user

router = APIRouter()

@router.get("/", response_model=list[PostResponse])
async def read_posts(skip: int = 0, limit: int = 100, db: Session = Depends(get_db)):
    """
    Vrátí seznam článků
    """
    posts = db.query(Post).offset(skip).limit(limit).all()
    return posts

@router.get("/{post_id}", response_model=PostResponse)
async def read_post(post_id: int, db: Session = Depends(get_db)):
    """
    Vrátí konkrétní článek podle ID
    """
    post = db.query(Post).filter(Post.id == post_id).first()
    
    if not post:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Post not found"
        )
        
    return post

@router.post("/", response_model=PostResponse)
async def create_post(
    post: PostCreate, 
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user)
):
    """
    Vytvoří nový článek
    """
    db_post = Post(**post.dict(), author_id=current_user.id)
    db.add(db_post)
    db.commit()
    db.refresh(db_post)
    
    return db_post

@router.put("/{post_id}", response_model=PostResponse)
async def update_post(
    post_id: int,
    post_update: PostUpdate,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user)
):
    """
    Aktualizuje článek
    """
    db_post = db.query(Post).filter(Post.id == post_id).first()
    
    if not db_post:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Post not found"
        )
        
    # Kontrola oprávnění - pouze autor nebo admin může upravovat
    if db_post.author_id != current_user.id and not current_user.is_superuser:
        raise HTTPException(
            status_code=status.HTTP_403_FORBIDDEN,
            detail="Not authorized to update this post"
        )
        
    for key, value in post_update.dict(exclude_unset=True).items():
        setattr(db_post, key, value)
        
    db.commit()
    db.refresh(db_post)
    
    return db_post

@router.delete("/{post_id}")
async def delete_post(
    post_id: int,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user)
):
    """
    Smaže článek
    """
    db_post = db.query(Post).filter(Post.id == post_id).first()
    
    if not db_post:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Post not found"
        )
        
    # Kontrola oprávnění - pouze autor nebo admin může smazat
    if db_post.author_id != current_user.id and not current_user.is_superuser:
        raise HTTPException(
            status_code=status.HTTP_403_FORBIDDEN,
            detail="Not authorized to delete this post"
        )
        
    db.delete(db_post)
    db.commit()
    
    return {"message": "Post deleted successfully"}
```

### app/api/v1/routes/users.py
```python
"""
Routy pro správu uživatelů
"""

from fastapi import APIRouter, HTTPException, status, Depends
from sqlalchemy.orm import Session

from app.core.database import get_db
from app.models.user import User
from app.api.v1.schemas.user import UserCreate, UserResponse, UserLogin
from app.api.v1.schemas.token import Token
from app.core.security import (
    verify_password, 
    get_password_hash, 
    create_access_token,
    get_current_user
)

router = APIRouter()

@router.post("/register", response_model=UserResponse)
async def register_user(user: UserCreate, db: Session = Depends(get_db)):
    """
    Registrace nového uživatele
    """
    # Kontrola, zda uživatel s tímto emailem již existuje
    db_user = db.query(User).filter(User.email == user.email).first()
    
    if db_user:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Email already registered"
        )
        
    # Vytvoření nového uživatele
    hashed_password = get_password_hash(user.password)
    
    db_user = User(
        email=user.email,
        username=user.username,
        hashed_password=hashed_password
    )
    
    db.add(db_user)
    db.commit()
    db.refresh(db_user)
    
    return db_user

@router.post("/login", response_model=Token)
async def login_for_access_token(
    user_login: UserLogin, 
    db: Session = Depends(get_db)
):
    """
    Přihlášení uživatele a vytvoření tokenu
    """
    db_user = db.query(User).filter(User.email == user_login.email).first()
    
    if not db_user or not verify_password(user_login.password, db_user.hashed_password):
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Incorrect email or password",
            headers={"WWW-Authenticate": "Bearer"},
        )
        
    access_token_expires = timedelta(minutes=30)
    access_token = create_access_token(
        data={"sub": db_user.email}, expires_delta=access_token_expires
    )
    
    return {"access_token": access_token, "token_type": "bearer"}

@router.get("/me", response_model=UserResponse)
async def read_users_me(current_user: User = Depends(get_current_user)):
    """
    Vrátí informace o přihlášeném uživateli
    """
    return current_user

@router.get("/{user_id}", response_model=UserResponse)
async def read_user(user_id: int, db: Session = Depends(get_db)):
    """
    Vrátí informace o konkrétním uživateli podle ID
    """
    user = db.query(User).filter(User.id == user_id).first()
    
    if not user:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="User not found"
        )
        
    return user

@router.delete("/{user_id}")
async def delete_user(
    user_id: int,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user)
):
    """
    Smaže


## 📚 Další zdroje a materiály

<div style="background: #e3f2fd; padding: 20px; border-left: 5px solid #2196f3; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🔗 Doporučené materiály k dalšímu studiu</h3>
</div>

# Zdroje pro kapitolu: Projekt 2 - REST API pro blog

## 1. Doporučené články / tutoriály

1. **[Building a REST API with Node.js and Express](https://www.freecodecamp.org/news/rest-api-tutorial/)**
   - Článek od FreeCodeCampu, který vás provede tvorbou REST API pomocí Node.js a Expressu. Obsahuje praktické příklady, popis HTTP metod a práce s databází.

2. **[REST API Design Best Practices](https://www.restapitutorial.com/)**
   - Komplexní průvodce pro návrh REST API, který zahrnuje konvence, kódování, autentizaci a řízení verzí.

3. **[Create a Blog API with Django REST Framework](https://www.django-rest-framework.org/tutorial/quickstart/)**
   - Kurz pro tvorbu REST API v Pythonu pomocí Django REST Frameworku. Vhodný pro začátečníky, kteří chtějí použít Python a Django.

4. **[Building a REST API with Spring Boot](https://spring.io/guides/gs/rest-service/)**
   - Oficiální průvodce od Springu pro vytváření REST API pomocí Java/Spring Boot. Přehledný a praktický návod s příklady.

5. **[REST API Design and Development with Flask](https://flask.palletsprojects.com/en/2.3.x/)**
   - Průvodce pro tvorbu REST API v Pythonu pomocí Flasku. Zahrnuje autentizaci, serializaci a testování.

## 2. YouTube videa

1. **[REST API Crash Course (Node.js, Express)](https://www.youtube.com/watch?v=O3yN5H7n19o) – délka: 45 min**
   - Výukové video od Programming with Mosh. Zabývá se tvorbou REST API v Node.js s využitím Expressu, včetně CRUD operací.

2. **[Build a Blog REST API with Django (Full Course)](https://www.youtube.com/watch?v=9JiX95h3wYU) – délka: 1 hodina**
   - Kompletní kurz pro tvorbu blogového REST API pomocí Django. Vhodný pro začátečníky, kteří chtějí zvládnout vše od návrhu po nasazení.

3. **[REST API with Java Spring Boot - Full Tutorial](https://www.youtube.com/watch?v=J2Q9Z4g4F3E) – délka: 1 hodina 30 minut**
   - Detailní video tutoriál o tvorbě REST API pomocí Spring Bootu. Zahrnuje vše od základů až po pokročilé koncepty jako transakce a validace.

## 3. Knihy / Oficiální dokumentace

1. **[REST API Design Best Practices – by Brian Mulloy](https://www.amazon.com/REST-API-Design-Best-Practices/dp/149205768X)**
   - Kniha se zaměřuje na osvědčené postupy při návrhu REST API. Obsahuje praktické tipy a vzory, které pomohou vytvořit čisté a udržitelné rozhraní.

2. **[Django REST Framework Documentation](https://www.django-rest-framework.org/)**
   - Oficiální dokumentace k Django REST Frameworku. Obsahuje podrobný průvodce používáním knihovny pro tvorbu API v Pythonu.

3. **[Spring Boot Documentation](https://docs.spring.io/spring-boot/docs/current/reference/html/)**
   - Kompletní dokumentace Spring Bootu, která zahrnuje i části o tvorbě REST API a práci s HTTP.

## 4. Praktické projekty

1. **[Build a Blog API with Node.js and MongoDB](https://github.com/avneesh0612/blog-api)**
   - Repozitář s komple


## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ FastAPI application structure</li>
<li>✓ User registration/authentication</li>
<li>✓ Blog posts CRUD</li>
<li>✓ Comments system</li>
<li>✓ Pagination a filtering</li>
    </ul>
    
    <h3 style="color: white;">🎯 Klíčové dovednosti</h3>
    <p>Po dokončení této kapitoly byste měli být schopni prakticky použít všechny probrané koncepty.</p>
    
    <h3 style='color: white;'>➡️ Další kapitola</h3><p>Kapitola 21 - pokračujte ve studiu!</p>
</div>

---

*📅 Notebook vygenerován: 2025-09-29 12:39:41*  
*🤖 Generátor: Comprehensive Colab Generator v2.0*  
*📚 Učebnice programování - Od základů k AI*


## 🧪 Sandbox - Prostor pro experimenty

Použijte následující buňky pro vlastní experimenty a testování:

In [ ]:
# 🧪 Zde můžete experimentovat s kódem z kapitoly
# Napište svůj kód zde:

